# Imports

In [ ]:
import tensorflow as tf
import os
import numpy as np
import math, numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras
from tensorflow.keras.optimizers import RMSprop
from pathlib import Path

from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML, display, clear_output, SVG

In [ ]:
Path('/content/weights').mkdir(parents=True, exist_ok=True)

# Setting up dataset

In [ ]:
# Download images from github
!git clone https://github.com/lukekoko/42028-Deep-Learning-and-Convolutional-Neural-Network-Assignment-3.git

In [ ]:
# get paths for directories
base_dir = './42028-Deep-Learning-and-Convolutional-Neural-Network-Assignment-3/dataset/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Directory with our training covid pictures
train_covid_dir = os.path.join(train_dir, 'covid19')

# Directory with our training normal pictures
train_normal_dir = os.path.join(train_dir, 'normal')

# Directory with our test covid pictures
test_covid_dir = os.path.join(test_dir, 'covid19')

# Directory with our test normal pictures
test_normal_dir = os.path.join(test_dir, 'normal')

In [ ]:
# data generation
train_datagen = ImageDataGenerator(rescale=1/255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   fill_mode='nearest', 
                                   zoom_range=0.2, 
                                   validation_split=0.2)
# validation_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1/255)

test_dataset = test_datagen.flow_from_directory(batch_size=32,
                                                directory=test_dir,
                                                shuffle=True,
                                                target_size=(150,150),
                                                class_mode='binary')

training_dataset = train_datagen.flow_from_directory(batch_size=32,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(150,150),
                                                subset="training",
                                                class_mode='binary')

validation_dataset = train_datagen.flow_from_directory(batch_size=32,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(150,150),
                                                subset="validation",
                                                class_mode='binary')



# Inception

In [ ]:
weightFilepath = '/content/weights'

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras import models
from keras import layers
from keras import optimizers

conv_base = InceptionV3(weights='imagenet',include_top=False, input_shape=(150, 150, 3))

model_inception = models.Sequential()
model_inception.add(conv_base)
model_inception.add(layers.Flatten())
model_inception.add(layers.Dense(256, activation='relu'))
model_inception.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
for layer in conv_base.layers[:-4]:
    layer.trainable = True
 
# Check the trainable status of the individual layers
# for layer in conv_base.layers:
#     print(layer, layer.trainable)

In [ ]:
model_inception.summary()

plot_model(model_inception, to_file='Inectpion.png', show_layer_names=True, show_shapes=True, rankdir='TB')
SVG(model_to_dot(model_inception, show_layer_names=True, show_shapes=True, rankdir='TB').create(prog='dot', format='svg'))

In [ ]:
model_inception.compile(loss='binary_crossentropy',
              # optimizer='adam',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
# callback checkpoint

filepath = weightFilepath + '/weights_{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')

In [ ]:
history_inception = model_inception.fit(
      training_dataset,
      epochs=10,
      validation_data=validation_dataset,
      callbacks = [checkpoint],
      verbose=1)

In [ ]:
model_inception.evaluate(test_dataset)

In [ ]:
model_inception.save(weightFilepath + '/model1save.hdf5')

In [ ]:
import matplotlib.pyplot as plt
acc = history_inception.history['acc']
val_acc = history_inception.history['val_acc']
loss = history_inception.history['loss']
val_loss = history_inception.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
labelArray = dict((v,k) for k,v in training_dataset.class_indices.items())

In [ ]:
labelArray

In [ ]:
imagesForPrediction = test_dataset
x, y = imagesForPrediction.next()

In [ ]:
# Predictions on images from test set using baseline pretrained
for i in range(0,32):
    image = x[i]
    label = labelArray[y[i]]
    prediction = model_inception.predict_classes(np.expand_dims(x[i], axis=0))
    predLabel = labelArray[prediction[0][0]]
    plt.imshow(image)
    plt.title("Prediction: {}, Actual: {}".format(predLabel, label))
    plt.axis('off')
    plt.show()

# Resnet

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
resnet_conv_base = ResNet50(weights='imagenet',include_top=False, input_shape=(150,150, 3))

for layer in resnet_conv_base.layers[:-4]:
    layer.trainable = False
    
model_resnet = models.Sequential()
model_resnet.add(resnet_conv_base)
model_resnet.add(layers.Flatten())
model_resnet.add(layers.Dense(256, activation='relu'))
model_resnet.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model_resnet.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
history_resnet = model_resnet.fit(
      training_dataset,
      # steps_per_epoch=2,  # 2000 images = batch_size * steps
      epochs=10,
      validation_data=validation_dataset,
      # validation_steps=50,  # 1000 images = batch_size * steps
      verbose=1)

In [ ]:
model_resnet.evaluate(test_dataset)

# VGG16

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg16_conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(150,150, 3))

for layer in vgg16_conv_base.layers[:-4]:
    layer.trainable = False
    
model_vgg16 = models.Sequential()
model_vgg16.add(resnet_conv_base)
model_vgg16.add(layers.Flatten())
model_vgg16.add(layers.Dense(256, activation='relu'))
model_vgg16.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model_vgg16.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
history_vgg16 = model_vgg16.fit(
      training_dataset,
      # steps_per_epoch=2,  # 2000 images = batch_size * steps
      epochs=10,
      validation_data=validation_dataset,
      # validation_steps=50,  # 1000 images = batch_size * steps
      verbose=1)

In [ ]:
model_vgg16.evaluate(test_dataset)

# AlexNet

In [ ]:
# data generation
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)
validation_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1/255)

test_dataset_alexnet = test_datagen.flow_from_directory(batch_size=20,
                                                directory=test_dir,
                                                shuffle=True,
                                                target_size=(224,224),
                                                class_mode='binary')

training_dataset_alexnet = train_datagen.flow_from_directory(batch_size=20,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(224,224),
                                                subset="training",
                                                class_mode='binary')

validation_dataset_alexnet = train_datagen.flow_from_directory(batch_size=20,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(224,224),
                                                subset="validation",
                                                class_mode='binary')



In [ ]:
model_alexnet = tf.keras.models.Sequential([
    #Conv_1          #original model was built for input shape of 224X224
    tf.keras.layers.Conv2D(96, (11,11),strides=4, padding='valid', activation='relu', input_shape=(224, 224, 3)),
    # Pooling_1
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2),padding='valid'),
    # Batch Normalisation_1
    tf.keras.layers.BatchNormalization(),
    # Conv_2
    tf.keras.layers.Conv2D(256, (5,5),strides=1, padding='valid', activation='relu'),
    # Pooling_2
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2),padding='valid'),
    #Batch Normalisation_2
    tf.keras.layers.BatchNormalization(),
    # Conv_3
    tf.keras.layers.Conv2D(384, (3,3),strides=1, padding='valid', activation='relu'),
    # Batch Normalisation_3
    tf.keras.layers.BatchNormalization(),
    # Conv_4
    tf.keras.layers.Conv2D(384, (3,3),strides=1, padding='valid', activation='relu'),
    # Batch Normalisation_3
    tf.keras.layers.BatchNormalization(),
    #conv_5
    tf.keras.layers.Conv2D(256, (3,3),strides=1, padding='valid', activation='relu'),
    #pooling_3
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2),padding='valid'),
    #Batch Normalization_4
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    #Dense layer_1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    #Dense layer_2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    #Dense layer_3
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])


In [ ]:
model_alexnet.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
history_alexnet = model_alexnet.fit(
      training_dataset_alexnet,
      #steps_per_epoch=1000,  # 2000 images = batch_size * steps
      epochs=10,
      validation_data=validation_dataset_alexnet,
      # validation_steps=50,  # 1000 images = batch_size * steps
      verbose=1)

In [ ]:
import matplotlib.pyplot as plt
acc = history_alexnet.history['acc']
val_acc = history_alexnet.history['val_acc']
loss = history_alexnet.history['loss']
val_loss = history_alexnet.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model_alexnet.evaluate(test_dataset_alexnet)